In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Load the data
train_data = np.load('./layer_features_base_100/train_3.npz')
test_data = np.load('./layer_features_base_100/test_3.npz')

train_features = train_data['features']
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# Fix input shape for CNN (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


Epoch 1/50


/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [2]:
import numpy as np
data = np.load('./layer_npz_features/train_3.npz')
features = data['features']
speaker_ids = data['speaker_ids']

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# Load the data

train_data = np.load('./layer_npz_features/train_3.npz')
test_data = np.load('./layer_npz_features/test_3.npz')

train_features = train_data['features']
train_labels = pd.read_csv('y_train.csv').Gender_bin
test_features = test_data['features']
test_labels = pd.read_csv('y_test.csv').Gender_bin

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)

# Transform only the filtered test labels
test_labels_encoded = label_encoder.transform(test_labels)

from sklearn.metrics import classification_report

from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)

svm_model.fit(train_features, train_labels_encoded)

svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# cnn_model = Sequential([
#     Conv1D(32, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
#     MaxPooling1D(pool_size=2),
#     Conv1D(128, kernel_size=5, activation='relu'),
#     MaxPooling1D(pool_size=2),
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(len(label_encoder.classes_), activation='softmax')
# ])

cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(1024, 1)),  # Input shape for CNN
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch+1}: Loss = {logs['loss']:.4f}, Accuracy = {logs['accuracy']:.4f}, Validation Loss = {logs['val_loss']:.4f}, Validation Accuracy = {logs['val_accuracy']:.4f}")

history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2
)

for epoch, (loss, accuracy, val_loss, val_accuracy) in enumerate(zip(
    history.history['loss'],
    history.history['accuracy'],
    history.history['val_loss'],
    history.history['val_accuracy']
)):
    print(f"Epoch {epoch+1}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}, Validation Loss = {val_loss:.4f}, Validation Accuracy = {val_accuracy:.4f}")

cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

from sklearn.metrics import classification_report

# Predict class probabilities
test_predictions_prob = cnn_model.predict(test_features_cnn)

# Get the class with the highest probability
test_predictions = np.argmax(test_predictions_prob, axis=1)

# Generate the classification report
report = classification_report(test_labels_encoded, test_predictions)
print(report)

cnn_model.summary()

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import LabelEncoder, StandardScaler

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# ann_model = Sequential([
#     Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
#     Dropout(0.5),
#     Dense(256, activation='relu'),
#     Dropout(0.5),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(len(label_encoder.classes_), activation='softmax')
# ])

ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(1024,)),  # Input shape for ANN
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2
)

for epoch, (loss, accuracy, val_loss, val_accuracy) in enumerate(zip(
    history.history['loss'],
    history.history['accuracy'],
    history.history['val_loss'],
    history.history['val_accuracy']
)):
    print(f"Epoch {epoch+1}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}, Validation Loss = {val_loss:.4f}, Validation Accuracy = {val_accuracy:.4f}")

ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

ann_model.summary()

import torch
from torch.utils.data import DataLoader, TensorDataset

# Convert NumPy arrays to PyTorch tensors with proper shape
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # (batch_size, sequence_length, channels) -> (batch_size, channels, sequence_length)
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

# Create datasets and data loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Define the CNN model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 1024, 512)  # Adjust based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Create tensors and data loaders
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize model, criterion, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

    def test_model(model, test_loader, criterion):
        model.eval()  # Set the model to evaluation mode
        test_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():  # No need to calculate gradients
            for inputs, labels in test_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                test_loss += loss.item() * inputs.size(0)  # Accumulate loss
                _, predicted = torch.max(outputs, 1)  # Get the index of the max log-probability
                total += labels.size(0)
                correct += (predicted == labels).sum().item()  # Count correct predictions

        avg_loss = test_loss / total
        accuracy = correct / total
        return avg_loss, accuracy


# Test the model
test_loss, test_accuracy = test_model(model, test_loader, criterion)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")



SVM Accuracy: 0.9457
SVM Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.93      0.94        56
           1       0.95      0.96      0.95        73

    accuracy                           0.95       129
   macro avg       0.95      0.94      0.94       129
weighted avg       0.95      0.95      0.95       129

Epoch 1/50


/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 463ms/step - accuracy: 0.5818 - loss: 2.2399 - val_accuracy: 0.7267 - val_loss: 0.4584
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 453ms/step - accuracy: 0.9508 - loss: 0.1572 - val_accuracy: 0.9477 - val_loss: 0.1663
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 454ms/step - accuracy: 0.9843 - loss: 0.0610 - val_accuracy: 0.9244 - val_loss: 0.2451
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 456ms/step - accuracy: 0.9939 - loss: 0.0278 - val_accuracy: 0.9012 - val_loss: 0.2727
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 455ms/step - accuracy: 0.9970 - loss: 0.0052 - val_accuracy: 0.9419 - val_loss: 0.1450
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 455ms/step - accuracy: 0.9996 - loss: 0.0026 - val_accuracy: 0.8547 - val_loss: 0.4788
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 456ms/step - accuracy: 0.9956 - loss: 0.0107 - val_accuracy: 0.9012 - val_loss: 0.2461
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 456ms/step - accuracy: 1.0000 - loss: 0.0024 - val_accuracy: 0.941

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_3 (Conv1D)               │ (None, 1020, 64)       │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 1016, 128)      │        41,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 1012, 256)      │       164,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 259072)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │    66,322,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

 Total params: 2,067,848 (7.89 MB)

 Trainable params: 689,282 (2.63 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,378,566 (5.26 MB)

Epoch 1: Loss = 16.7249
Epoch 2: Loss = 1.7317
Epoch 3: Loss = 0.4645
Epoch 4: Loss = 0.0723
Epoch 5: Loss = 0.0250
Epoch 6: Loss = 0.0209
Epoch 7: Loss = 0.0112
Epoch 8: Loss = 0.0187
Epoch 9: Loss = 0.0064
Epoch 10: Loss = 0.0036
Epoch 11: Loss = 0.0028
Epoch 12: Loss = 0.0025
Epoch 13: Loss = 0.0021
Epoch 14: Loss = 0.0026
Epoch 15: Loss = 0.0058
Epoch 16: Loss = 0.0054
Epoch 17: Loss = 0.0051
Epoch 18: Loss = 0.0029
Epoch 19: Loss = 0.0080
Epoch 20: Loss = 0.0055
Epoch 21: Loss = 0.0088
Epoch 22: Loss = 0.0042
Epoch 23: Loss = 0.0010
Epoch 24: Loss = 0.0008
Epoch 25: Loss = 0.0007
Epoch 26: Loss = 0.0002
Epoch 27: Loss = 0.0008
Epoch 28: Loss = 0.0002
Epoch 29: Loss = 0.0010
Epoch 30: Loss = 0.0012
Epoch 31: Loss = 0.0009
Epoch 32: Loss = 0.0004
Epoch 33: Loss = 0.0002
Epoch 34: Loss = 0.0004
Epoch 35: Loss = 0.0001
Epoch 36: Loss = 0.0001
Epoch 37: Loss = 0.0017
Epoch 38: Loss = 0.0020
Epoch 39: Loss = 0.0011
Epoch 40: Loss = 0.0005
Epoch 41: Loss = 0.0003
Epoch 42: Loss = 0.0008


In [3]:
import numpy as np
data = np.load('./layer_npz_features/train_4.npz')
features = data['features']
speaker_ids = data['speaker_ids']

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# Load the data

train_data = np.load('./layer_npz_features/train_4.npz')
test_data = np.load('./layer_npz_features/test_4.npz')

train_features = train_data['features']
train_labels = pd.read_csv('y_train.csv').Gender_bin
test_features = test_data['features']
test_labels = pd.read_csv('y_test.csv').Gender_bin

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)

# Transform only the filtered test labels
test_labels_encoded = label_encoder.transform(test_labels)

from sklearn.metrics import classification_report

from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)

svm_model.fit(train_features, train_labels_encoded)

svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# cnn_model = Sequential([
#     Conv1D(32, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
#     MaxPooling1D(pool_size=2),
#     Conv1D(128, kernel_size=5, activation='relu'),
#     MaxPooling1D(pool_size=2),
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(len(label_encoder.classes_), activation='softmax')
# ])

cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(1024, 1)),  # Input shape for CNN
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch+1}: Loss = {logs['loss']:.4f}, Accuracy = {logs['accuracy']:.4f}, Validation Loss = {logs['val_loss']:.4f}, Validation Accuracy = {logs['val_accuracy']:.4f}")

history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2
)

for epoch, (loss, accuracy, val_loss, val_accuracy) in enumerate(zip(
    history.history['loss'],
    history.history['accuracy'],
    history.history['val_loss'],
    history.history['val_accuracy']
)):
    print(f"Epoch {epoch+1}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}, Validation Loss = {val_loss:.4f}, Validation Accuracy = {val_accuracy:.4f}")

cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

from sklearn.metrics import classification_report

# Predict class probabilities
test_predictions_prob = cnn_model.predict(test_features_cnn)

# Get the class with the highest probability
test_predictions = np.argmax(test_predictions_prob, axis=1)

# Generate the classification report
report = classification_report(test_labels_encoded, test_predictions)
print(report)

cnn_model.summary()

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import LabelEncoder, StandardScaler

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# ann_model = Sequential([
#     Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
#     Dropout(0.5),
#     Dense(256, activation='relu'),
#     Dropout(0.5),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(len(label_encoder.classes_), activation='softmax')
# ])

ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(1024,)),  # Input shape for ANN
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2
)

for epoch, (loss, accuracy, val_loss, val_accuracy) in enumerate(zip(
    history.history['loss'],
    history.history['accuracy'],
    history.history['val_loss'],
    history.history['val_accuracy']
)):
    print(f"Epoch {epoch+1}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}, Validation Loss = {val_loss:.4f}, Validation Accuracy = {val_accuracy:.4f}")

ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

ann_model.summary()

import torch
from torch.utils.data import DataLoader, TensorDataset

# Convert NumPy arrays to PyTorch tensors with proper shape
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # (batch_size, sequence_length, channels) -> (batch_size, channels, sequence_length)
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

# Create datasets and data loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Define the CNN model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 1024, 512)  # Adjust based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Create tensors and data loaders
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize model, criterion, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

    def test_model(model, test_loader, criterion):
        model.eval()  # Set the model to evaluation mode
        test_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():  # No need to calculate gradients
            for inputs, labels in test_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                test_loss += loss.item() * inputs.size(0)  # Accumulate loss
                _, predicted = torch.max(outputs, 1)  # Get the index of the max log-probability
                total += labels.size(0)
                correct += (predicted == labels).sum().item()  # Count correct predictions

        avg_loss = test_loss / total
        accuracy = correct / total
        return avg_loss, accuracy


# Test the model
test_loss, test_accuracy = test_model(model, test_loader, criterion)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")



SVM Accuracy: 0.9380
SVM Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.91      0.93        56
           1       0.93      0.96      0.95        73

    accuracy                           0.94       129
   macro avg       0.94      0.93      0.94       129
weighted avg       0.94      0.94      0.94       129

Epoch 1/50


/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 477ms/step - accuracy: 0.5832 - loss: 2.5420 - val_accuracy: 0.9477 - val_loss: 0.2549
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 464ms/step - accuracy: 0.9404 - loss: 0.1511 - val_accuracy: 0.8953 - val_loss: 0.2647
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 458ms/step - accuracy: 0.9824 - loss: 0.0587 - val_accuracy: 0.9477 - val_loss: 0.1147
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 459ms/step - accuracy: 0.9883 - loss: 0.0319 - val_accuracy: 0.9709 - val_loss: 0.0697
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 459ms/step - accuracy: 0.9929 - loss: 0.0135 - val_accuracy: 0.9419 - val_loss: 0.1703
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 459ms/step - accuracy: 0.9966 - loss: 0.0151 - val_accuracy: 0.9360 - val_loss: 0.1921
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 458ms/step - accuracy: 1.0000 - loss: 0.0031 - val_accuracy: 0.9477 - val_loss: 0.2055
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 462ms/step - accuracy: 0.9970 - loss: 0.0070 - val_accuracy: 0.947

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_6 (Conv1D)               │ (None, 1020, 64)       │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_7 (Conv1D)               │ (None, 1016, 128)      │        41,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_8 (Conv1D)               │ (None, 1012, 256)      │       164,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 259072)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 256)            │    66,322,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

 Total params: 2,067,848 (7.89 MB)

 Trainable params: 689,282 (2.63 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,378,566 (5.26 MB)

Epoch 1: Loss = 16.9157
Epoch 2: Loss = 2.3778
Epoch 3: Loss = 0.4730
Epoch 4: Loss = 0.2578
Epoch 5: Loss = 0.0973
Epoch 6: Loss = 0.0356
Epoch 7: Loss = 0.0322
Epoch 8: Loss = 0.0091
Epoch 9: Loss = 0.0117
Epoch 10: Loss = 0.0029
Epoch 11: Loss = 0.0015
Epoch 12: Loss = 0.0026
Epoch 13: Loss = 0.0013
Epoch 14: Loss = 0.0006
Epoch 15: Loss = 0.0023
Epoch 16: Loss = 0.0164
Epoch 17: Loss = 0.0053
Epoch 18: Loss = 0.0164
Epoch 19: Loss = 0.0148
Epoch 20: Loss = 0.0020
Epoch 21: Loss = 0.0030
Epoch 22: Loss = 0.0005
Epoch 23: Loss = 0.0003
Epoch 24: Loss = 0.0001
Epoch 25: Loss = 0.0002
Epoch 26: Loss = 0.0004
Epoch 27: Loss = 0.0007
Epoch 28: Loss = 0.0004
Epoch 29: Loss = 0.0114
Epoch 30: Loss = 0.0026
Epoch 31: Loss = 0.0716
Epoch 32: Loss = 0.0651
Epoch 33: Loss = 0.0185
Epoch 34: Loss = 0.0265
Epoch 35: Loss = 0.0005
Epoch 36: Loss = 0.0093
Epoch 37: Loss = 0.0042
Epoch 38: Loss = 0.0029
Epoch 39: Loss = 0.0039
Epoch 40: Loss = 0.0290
Epoch 41: Loss = 0.0083
Epoch 42: Loss = 0.0003


In [4]:
import numpy as np
data = np.load('./layer_npz_features/train_6.npz')
features = data['features']
speaker_ids = data['speaker_ids']

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# Load the data

train_data = np.load('./layer_npz_features/train_6.npz')
test_data = np.load('./layer_npz_features/test_6.npz')

train_features = train_data['features']
train_labels = pd.read_csv('y_train.csv').Gender_bin
test_features = test_data['features']
test_labels = pd.read_csv('y_test.csv').Gender_bin

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)

# Transform only the filtered test labels
test_labels_encoded = label_encoder.transform(test_labels)

from sklearn.metrics import classification_report

from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)

svm_model.fit(train_features, train_labels_encoded)

svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# cnn_model = Sequential([
#     Conv1D(32, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
#     MaxPooling1D(pool_size=2),
#     Conv1D(128, kernel_size=5, activation='relu'),
#     MaxPooling1D(pool_size=2),
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(len(label_encoder.classes_), activation='softmax')
# ])

cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(1024, 1)),  # Input shape for CNN
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch+1}: Loss = {logs['loss']:.4f}, Accuracy = {logs['accuracy']:.4f}, Validation Loss = {logs['val_loss']:.4f}, Validation Accuracy = {logs['val_accuracy']:.4f}")

history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2
)

for epoch, (loss, accuracy, val_loss, val_accuracy) in enumerate(zip(
    history.history['loss'],
    history.history['accuracy'],
    history.history['val_loss'],
    history.history['val_accuracy']
)):
    print(f"Epoch {epoch+1}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}, Validation Loss = {val_loss:.4f}, Validation Accuracy = {val_accuracy:.4f}")

cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

from sklearn.metrics import classification_report

# Predict class probabilities
test_predictions_prob = cnn_model.predict(test_features_cnn)

# Get the class with the highest probability
test_predictions = np.argmax(test_predictions_prob, axis=1)

# Generate the classification report
report = classification_report(test_labels_encoded, test_predictions)
print(report)

cnn_model.summary()

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import LabelEncoder, StandardScaler

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# ann_model = Sequential([
#     Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
#     Dropout(0.5),
#     Dense(256, activation='relu'),
#     Dropout(0.5),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(len(label_encoder.classes_), activation='softmax')
# ])

ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(1024,)),  # Input shape for ANN
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2
)

for epoch, (loss, accuracy, val_loss, val_accuracy) in enumerate(zip(
    history.history['loss'],
    history.history['accuracy'],
    history.history['val_loss'],
    history.history['val_accuracy']
)):
    print(f"Epoch {epoch+1}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}, Validation Loss = {val_loss:.4f}, Validation Accuracy = {val_accuracy:.4f}")

ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

ann_model.summary()

import torch
from torch.utils.data import DataLoader, TensorDataset

# Convert NumPy arrays to PyTorch tensors with proper shape
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # (batch_size, sequence_length, channels) -> (batch_size, channels, sequence_length)
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

# Create datasets and data loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Define the CNN model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 1024, 512)  # Adjust based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Create tensors and data loaders
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize model, criterion, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

    def test_model(model, test_loader, criterion):
        model.eval()  # Set the model to evaluation mode
        test_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():  # No need to calculate gradients
            for inputs, labels in test_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                test_loss += loss.item() * inputs.size(0)  # Accumulate loss
                _, predicted = torch.max(outputs, 1)  # Get the index of the max log-probability
                total += labels.size(0)
                correct += (predicted == labels).sum().item()  # Count correct predictions

        avg_loss = test_loss / total
        accuracy = correct / total
        return avg_loss, accuracy


# Test the model
test_loss, test_accuracy = test_model(model, test_loader, criterion)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")



SVM Accuracy: 0.9147
SVM Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.86      0.90        56
           1       0.90      0.96      0.93        73

    accuracy                           0.91       129
   macro avg       0.92      0.91      0.91       129
weighted avg       0.92      0.91      0.91       129

Epoch 1/50


/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 471ms/step - accuracy: 0.5761 - loss: 2.4578 - val_accuracy: 0.7267 - val_loss: 0.4811
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 458ms/step - accuracy: 0.8781 - loss: 0.2592 - val_accuracy: 0.9477 - val_loss: 0.1123
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 456ms/step - accuracy: 0.9636 - loss: 0.0916 - val_accuracy: 0.9302 - val_loss: 0.1815
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 456ms/step - accuracy: 0.9986 - loss: 0.0184 - val_accuracy: 0.8895 - val_loss: 0.2833
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 454ms/step - accuracy: 0.9915 - loss: 0.0173 - val_accuracy: 0.8488 - val_loss: 0.4562
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 452ms/step - accuracy: 0.9962 - loss: 0.0166 - val_accuracy: 0.8430 - val_loss: 0.3936
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 454ms/step - accuracy: 0.9976 - loss: 0.0124 - val_accuracy: 0.9128 - val_loss: 0.2208
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 451ms/step - accuracy: 1.0000 - loss: 0.0049 - val_accuracy: 0.924

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_9 (Conv1D)               │ (None, 1020, 64)       │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_10 (Conv1D)              │ (None, 1016, 128)      │        41,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_11 (Conv1D)              │ (None, 1012, 256)      │       164,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 259072)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 256)            │    66,322,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

 Total params: 2,067,848 (7.89 MB)

 Trainable params: 689,282 (2.63 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,378,566 (5.26 MB)

Epoch 1: Loss = 20.6942
Epoch 2: Loss = 3.6471
Epoch 3: Loss = 0.8746
Epoch 4: Loss = 0.1608
Epoch 5: Loss = 0.1305
Epoch 6: Loss = 0.0682
Epoch 7: Loss = 0.0266
Epoch 8: Loss = 0.0149
Epoch 9: Loss = 0.0092
Epoch 10: Loss = 0.0101
Epoch 11: Loss = 0.0047
Epoch 12: Loss = 0.0037
Epoch 13: Loss = 0.0056
Epoch 14: Loss = 0.0039
Epoch 15: Loss = 0.0026
Epoch 16: Loss = 0.0021
Epoch 17: Loss = 0.0061
Epoch 18: Loss = 0.0030
Epoch 19: Loss = 0.0035
Epoch 20: Loss = 0.0015
Epoch 21: Loss = 0.0031
Epoch 22: Loss = 0.0013
Epoch 23: Loss = 0.0014
Epoch 24: Loss = 0.0008
Epoch 25: Loss = 0.0012
Epoch 26: Loss = 0.0009
Epoch 27: Loss = 0.0007
Epoch 28: Loss = 0.0006
Epoch 29: Loss = 0.0003
Epoch 30: Loss = 0.0005
Epoch 31: Loss = 0.0007
Epoch 32: Loss = 0.0004
Epoch 33: Loss = 0.0004
Epoch 34: Loss = 0.0008
Epoch 35: Loss = 0.0003
Epoch 36: Loss = 0.0010
Epoch 37: Loss = 0.0010
Epoch 38: Loss = 0.0007
Epoch 39: Loss = 0.0006
Epoch 40: Loss = 0.0012
Epoch 41: Loss = 0.0010
Epoch 42: Loss = 0.0006


In [5]:
import numpy as np
data = np.load('./layer_npz_features/train_7.npz')
features = data['features']
speaker_ids = data['speaker_ids']

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# Load the data

train_data = np.load('./layer_npz_features/train_7.npz')
test_data = np.load('./layer_npz_features/test_7.npz')

train_features = train_data['features']
train_labels = pd.read_csv('y_train.csv').Gender_bin
test_features = test_data['features']
test_labels = pd.read_csv('y_test.csv').Gender_bin

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)

# Transform only the filtered test labels
test_labels_encoded = label_encoder.transform(test_labels)

from sklearn.metrics import classification_report

from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)

svm_model.fit(train_features, train_labels_encoded)

svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# cnn_model = Sequential([
#     Conv1D(32, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
#     MaxPooling1D(pool_size=2),
#     Conv1D(128, kernel_size=5, activation='relu'),
#     MaxPooling1D(pool_size=2),
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(len(label_encoder.classes_), activation='softmax')
# ])

cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(1024, 1)),  # Input shape for CNN
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch+1}: Loss = {logs['loss']:.4f}, Accuracy = {logs['accuracy']:.4f}, Validation Loss = {logs['val_loss']:.4f}, Validation Accuracy = {logs['val_accuracy']:.4f}")

history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2
)

for epoch, (loss, accuracy, val_loss, val_accuracy) in enumerate(zip(
    history.history['loss'],
    history.history['accuracy'],
    history.history['val_loss'],
    history.history['val_accuracy']
)):
    print(f"Epoch {epoch+1}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}, Validation Loss = {val_loss:.4f}, Validation Accuracy = {val_accuracy:.4f}")

cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

from sklearn.metrics import classification_report

# Predict class probabilities
test_predictions_prob = cnn_model.predict(test_features_cnn)

# Get the class with the highest probability
test_predictions = np.argmax(test_predictions_prob, axis=1)

# Generate the classification report
report = classification_report(test_labels_encoded, test_predictions)
print(report)

cnn_model.summary()

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import LabelEncoder, StandardScaler

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# ann_model = Sequential([
#     Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
#     Dropout(0.5),
#     Dense(256, activation='relu'),
#     Dropout(0.5),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(len(label_encoder.classes_), activation='softmax')
# ])

ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(1024,)),  # Input shape for ANN
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2
)

for epoch, (loss, accuracy, val_loss, val_accuracy) in enumerate(zip(
    history.history['loss'],
    history.history['accuracy'],
    history.history['val_loss'],
    history.history['val_accuracy']
)):
    print(f"Epoch {epoch+1}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}, Validation Loss = {val_loss:.4f}, Validation Accuracy = {val_accuracy:.4f}")

ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

ann_model.summary()

import torch
from torch.utils.data import DataLoader, TensorDataset

# Convert NumPy arrays to PyTorch tensors with proper shape
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # (batch_size, sequence_length, channels) -> (batch_size, channels, sequence_length)
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

# Create datasets and data loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Define the CNN model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 1024, 512)  # Adjust based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Create tensors and data loaders
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize model, criterion, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

    def test_model(model, test_loader, criterion):
        model.eval()  # Set the model to evaluation mode
        test_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():  # No need to calculate gradients
            for inputs, labels in test_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                test_loss += loss.item() * inputs.size(0)  # Accumulate loss
                _, predicted = torch.max(outputs, 1)  # Get the index of the max log-probability
                total += labels.size(0)
                correct += (predicted == labels).sum().item()  # Count correct predictions

        avg_loss = test_loss / total
        accuracy = correct / total
        return avg_loss, accuracy


# Test the model
test_loss, test_accuracy = test_model(model, test_loader, criterion)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")



SVM Accuracy: 0.9302
SVM Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.91      0.92        56
           1       0.93      0.95      0.94        73

    accuracy                           0.93       129
   macro avg       0.93      0.93      0.93       129
weighted avg       0.93      0.93      0.93       129

Epoch 1/50


/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 471ms/step - accuracy: 0.5891 - loss: 2.2748 - val_accuracy: 0.8547 - val_loss: 0.2941
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 460ms/step - accuracy: 0.8863 - loss: 0.2841 - val_accuracy: 0.9419 - val_loss: 0.1625
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 457ms/step - accuracy: 0.9717 - loss: 0.0832 - val_accuracy: 0.9012 - val_loss: 0.3170
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 460ms/step - accuracy: 0.9840 - loss: 0.0539 - val_accuracy: 0.8605 - val_loss: 0.3340
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 458ms/step - accuracy: 0.9824 - loss: 0.0645 - val_accuracy: 0.9593 - val_loss: 0.1125
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 459ms/step - accuracy: 0.9950 - loss: 0.0120 - val_accuracy: 0.9593 - val_loss: 0.1068
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 457ms/step - accuracy: 0.9965 - loss: 0.0117 - val_accuracy: 0.9477 - val_loss: 0.1208
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 460ms/step - accuracy: 0.9971 - loss: 0.0164 - val_accuracy: 0.941

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_12 (Conv1D)              │ (None, 1020, 64)       │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_13 (Conv1D)              │ (None, 1016, 128)      │        41,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_14 (Conv1D)              │ (None, 1012, 256)      │       164,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 259072)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 256)            │    66,322,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

 Total params: 2,067,848 (7.89 MB)

 Trainable params: 689,282 (2.63 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,378,566 (5.26 MB)

Epoch 1: Loss = 25.6549
Epoch 2: Loss = 2.5905
Epoch 3: Loss = 0.7434
Epoch 4: Loss = 0.1615
Epoch 5: Loss = 0.1035
Epoch 6: Loss = 0.0543
Epoch 7: Loss = 0.0517
Epoch 8: Loss = 0.0156
Epoch 9: Loss = 0.0183
Epoch 10: Loss = 0.0099
Epoch 11: Loss = 0.0075
Epoch 12: Loss = 0.0094
Epoch 13: Loss = 0.0072
Epoch 14: Loss = 0.0043
Epoch 15: Loss = 0.0034
Epoch 16: Loss = 0.0031
Epoch 17: Loss = 0.0031
Epoch 18: Loss = 0.0047
Epoch 19: Loss = 0.0030
Epoch 20: Loss = 0.0019
Epoch 21: Loss = 0.0012
Epoch 22: Loss = 0.0010
Epoch 23: Loss = 0.0010
Epoch 24: Loss = 0.0016
Epoch 25: Loss = 0.0009
Epoch 26: Loss = 0.0014
Epoch 27: Loss = 0.0008
Epoch 28: Loss = 0.0021
Epoch 29: Loss = 0.0010
Epoch 30: Loss = 0.0012
Epoch 31: Loss = 0.0008
Epoch 32: Loss = 0.0008
Epoch 33: Loss = 0.0015
Epoch 34: Loss = 0.0008
Epoch 35: Loss = 0.0006
Epoch 36: Loss = 0.0009
Epoch 37: Loss = 0.0014
Epoch 38: Loss = 0.0011
Epoch 39: Loss = 0.0005
Epoch 40: Loss = 0.0007
Epoch 41: Loss = 0.0005
Epoch 42: Loss = 0.0004


In [6]:
import numpy as np
data = np.load('./layer_npz_features/train_8.npz')
features = data['features']
speaker_ids = data['speaker_ids']

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# Load the data

train_data = np.load('./layer_npz_features/train_8.npz')
test_data = np.load('./layer_npz_features/test_8.npz')

train_features = train_data['features']
train_labels = pd.read_csv('y_train.csv').Gender_bin
test_features = test_data['features']
test_labels = pd.read_csv('y_test.csv').Gender_bin

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)

# Transform only the filtered test labels
test_labels_encoded = label_encoder.transform(test_labels)

from sklearn.metrics import classification_report

from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)

svm_model.fit(train_features, train_labels_encoded)

svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# cnn_model = Sequential([
#     Conv1D(32, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
#     MaxPooling1D(pool_size=2),
#     Conv1D(128, kernel_size=5, activation='relu'),
#     MaxPooling1D(pool_size=2),
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(len(label_encoder.classes_), activation='softmax')
# ])

cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(1024, 1)),  # Input shape for CNN
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch+1}: Loss = {logs['loss']:.4f}, Accuracy = {logs['accuracy']:.4f}, Validation Loss = {logs['val_loss']:.4f}, Validation Accuracy = {logs['val_accuracy']:.4f}")

history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2
)

for epoch, (loss, accuracy, val_loss, val_accuracy) in enumerate(zip(
    history.history['loss'],
    history.history['accuracy'],
    history.history['val_loss'],
    history.history['val_accuracy']
)):
    print(f"Epoch {epoch+1}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}, Validation Loss = {val_loss:.4f}, Validation Accuracy = {val_accuracy:.4f}")

cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

from sklearn.metrics import classification_report

# Predict class probabilities
test_predictions_prob = cnn_model.predict(test_features_cnn)

# Get the class with the highest probability
test_predictions = np.argmax(test_predictions_prob, axis=1)

# Generate the classification report
report = classification_report(test_labels_encoded, test_predictions)
print(report)

cnn_model.summary()

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import LabelEncoder, StandardScaler

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# ann_model = Sequential([
#     Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
#     Dropout(0.5),
#     Dense(256, activation='relu'),
#     Dropout(0.5),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(len(label_encoder.classes_), activation='softmax')
# ])

ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(1024,)),  # Input shape for ANN
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2
)

for epoch, (loss, accuracy, val_loss, val_accuracy) in enumerate(zip(
    history.history['loss'],
    history.history['accuracy'],
    history.history['val_loss'],
    history.history['val_accuracy']
)):
    print(f"Epoch {epoch+1}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}, Validation Loss = {val_loss:.4f}, Validation Accuracy = {val_accuracy:.4f}")

ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

ann_model.summary()

import torch
from torch.utils.data import DataLoader, TensorDataset

# Convert NumPy arrays to PyTorch tensors with proper shape
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # (batch_size, sequence_length, channels) -> (batch_size, channels, sequence_length)
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

# Create datasets and data loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Define the CNN model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 1024, 512)  # Adjust based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Create tensors and data loaders
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize model, criterion, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

    def test_model(model, test_loader, criterion):
        model.eval()  # Set the model to evaluation mode
        test_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():  # No need to calculate gradients
            for inputs, labels in test_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                test_loss += loss.item() * inputs.size(0)  # Accumulate loss
                _, predicted = torch.max(outputs, 1)  # Get the index of the max log-probability
                total += labels.size(0)
                correct += (predicted == labels).sum().item()  # Count correct predictions

        avg_loss = test_loss / total
        accuracy = correct / total
        return avg_loss, accuracy


# Test the model
test_loss, test_accuracy = test_model(model, test_loader, criterion)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")



SVM Accuracy: 0.9380
SVM Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.96      0.93        56
           1       0.97      0.92      0.94        73

    accuracy                           0.94       129
   macro avg       0.94      0.94      0.94       129
weighted avg       0.94      0.94      0.94       129

Epoch 1/50


/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 12s 469ms/step - accuracy: 0.5556 - loss: 4.2771 - val_accuracy: 0.8547 - val_loss: 0.5275
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 457ms/step - accuracy: 0.8027 - loss: 0.4734 - val_accuracy: 0.9070 - val_loss: 0.2735
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 454ms/step - accuracy: 0.9139 - loss: 0.2279 - val_accuracy: 0.9360 - val_loss: 0.1698
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 453ms/step - accuracy: 0.9639 - loss: 0.1104 - val_accuracy: 0.9360 - val_loss: 0.1456
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 455ms/step - accuracy: 0.9768 - loss: 0.0745 - val_accuracy: 0.9535 - val_loss: 0.1024
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 454ms/step - accuracy: 0.9856 - loss: 0.0460 - val_accuracy: 0.9419 - val_loss: 0.1139
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 453ms/step - accuracy: 0.9896 - loss: 0.0238 - val_accuracy: 0.9593 - val_loss: 0.1079
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 456ms/step - accuracy: 0.9855 - loss: 0.0401 - val_accuracy: 0.947

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_15 (Conv1D)              │ (None, 1020, 64)       │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_16 (Conv1D)              │ (None, 1016, 128)      │        41,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_17 (Conv1D)              │ (None, 1012, 256)      │       164,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 259072)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 256)            │    66,322,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

 Total params: 2,067,848 (7.89 MB)

 Trainable params: 689,282 (2.63 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,378,566 (5.26 MB)

Epoch 1: Loss = 19.4874
Epoch 2: Loss = 2.5074
Epoch 3: Loss = 0.7536
Epoch 4: Loss = 0.1803
Epoch 5: Loss = 0.1167
Epoch 6: Loss = 0.0619
Epoch 7: Loss = 0.0237
Epoch 8: Loss = 0.0141
Epoch 9: Loss = 0.0292
Epoch 10: Loss = 0.0230
Epoch 11: Loss = 0.0167
Epoch 12: Loss = 0.0120
Epoch 13: Loss = 0.0111
Epoch 14: Loss = 0.0091
Epoch 15: Loss = 0.0214
Epoch 16: Loss = 0.0146
Epoch 17: Loss = 0.0114
Epoch 18: Loss = 0.0113
Epoch 19: Loss = 0.0085
Epoch 20: Loss = 0.0089
Epoch 21: Loss = 0.0174
Epoch 22: Loss = 0.0267
Epoch 23: Loss = 0.0197
Epoch 24: Loss = 0.0769
Epoch 25: Loss = 0.0151
Epoch 26: Loss = 0.0225
Epoch 27: Loss = 0.0130
Epoch 28: Loss = 0.0058
Epoch 29: Loss = 0.0059
Epoch 30: Loss = 0.0057
Epoch 31: Loss = 0.0044
Epoch 32: Loss = 0.0032
Epoch 33: Loss = 0.0014
Epoch 34: Loss = 0.0054
Epoch 35: Loss = 0.0033
Epoch 36: Loss = 0.0018
Epoch 37: Loss = 0.0015
Epoch 38: Loss = 0.0009
Epoch 39: Loss = 0.0008
Epoch 40: Loss = 0.0007
Epoch 41: Loss = 0.0017
Epoch 42: Loss = 0.0005


In [7]:
import numpy as np
data = np.load('./layer_npz_features/train_9.npz')
features = data['features']
speaker_ids = data['speaker_ids']

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# Load the data

train_data = np.load('./layer_npz_features/train_9.npz')
test_data = np.load('./layer_npz_features/test_9.npz')

train_features = train_data['features']
train_labels = pd.read_csv('y_train.csv').Gender_bin
test_features = test_data['features']
test_labels = pd.read_csv('y_test.csv').Gender_bin

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)

# Transform only the filtered test labels
test_labels_encoded = label_encoder.transform(test_labels)

from sklearn.metrics import classification_report

from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)

svm_model.fit(train_features, train_labels_encoded)

svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# cnn_model = Sequential([
#     Conv1D(32, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
#     MaxPooling1D(pool_size=2),
#     Conv1D(128, kernel_size=5, activation='relu'),
#     MaxPooling1D(pool_size=2),
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(len(label_encoder.classes_), activation='softmax')
# ])

cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(1024, 1)),  # Input shape for CNN
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch+1}: Loss = {logs['loss']:.4f}, Accuracy = {logs['accuracy']:.4f}, Validation Loss = {logs['val_loss']:.4f}, Validation Accuracy = {logs['val_accuracy']:.4f}")

history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2
)

for epoch, (loss, accuracy, val_loss, val_accuracy) in enumerate(zip(
    history.history['loss'],
    history.history['accuracy'],
    history.history['val_loss'],
    history.history['val_accuracy']
)):
    print(f"Epoch {epoch+1}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}, Validation Loss = {val_loss:.4f}, Validation Accuracy = {val_accuracy:.4f}")

cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

from sklearn.metrics import classification_report

# Predict class probabilities
test_predictions_prob = cnn_model.predict(test_features_cnn)

# Get the class with the highest probability
test_predictions = np.argmax(test_predictions_prob, axis=1)

# Generate the classification report
report = classification_report(test_labels_encoded, test_predictions)
print(report)

cnn_model.summary()

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import LabelEncoder, StandardScaler

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# ann_model = Sequential([
#     Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
#     Dropout(0.5),
#     Dense(256, activation='relu'),
#     Dropout(0.5),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(len(label_encoder.classes_), activation='softmax')
# ])

ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(1024,)),  # Input shape for ANN
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2
)

for epoch, (loss, accuracy, val_loss, val_accuracy) in enumerate(zip(
    history.history['loss'],
    history.history['accuracy'],
    history.history['val_loss'],
    history.history['val_accuracy']
)):
    print(f"Epoch {epoch+1}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}, Validation Loss = {val_loss:.4f}, Validation Accuracy = {val_accuracy:.4f}")

ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

ann_model.summary()

import torch
from torch.utils.data import DataLoader, TensorDataset

# Convert NumPy arrays to PyTorch tensors with proper shape
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # (batch_size, sequence_length, channels) -> (batch_size, channels, sequence_length)
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

# Create datasets and data loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Define the CNN model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 1024, 512)  # Adjust based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Create tensors and data loaders
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize model, criterion, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

    def test_model(model, test_loader, criterion):
        model.eval()  # Set the model to evaluation mode
        test_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():  # No need to calculate gradients
            for inputs, labels in test_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                test_loss += loss.item() * inputs.size(0)  # Accumulate loss
                _, predicted = torch.max(outputs, 1)  # Get the index of the max log-probability
                total += labels.size(0)
                correct += (predicted == labels).sum().item()  # Count correct predictions

        avg_loss = test_loss / total
        accuracy = correct / total
        return avg_loss, accuracy


# Test the model
test_loss, test_accuracy = test_model(model, test_loader, criterion)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")



SVM Accuracy: 0.9225
SVM Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.96      0.92        56
           1       0.97      0.89      0.93        73

    accuracy                           0.92       129
   macro avg       0.92      0.93      0.92       129
weighted avg       0.93      0.92      0.92       129



/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 12s 515ms/step - accuracy: 0.6340 - loss: 1.0720 - val_accuracy: 0.8721 - val_loss: 0.3847
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 492ms/step - accuracy: 0.9069 - loss: 0.3000 - val_accuracy: 0.8953 - val_loss: 0.2732
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 486ms/step - accuracy: 0.9336 - loss: 0.1638 - val_accuracy: 0.9302 - val_loss: 0.1518
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 489ms/step - accuracy: 0.9894 - loss: 0.0482 - val_accuracy: 0.9651 - val_loss: 0.1108
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 492ms/step - accuracy: 0.9970 - loss: 0.0218 - val_accuracy: 0.9360 - val_loss: 0.1249
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 477ms/step - accuracy: 0.9918 - loss: 0.0357 - val_accuracy: 0.9360 - val_loss: 0.1417
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 489ms/step - accuracy: 0.9921 - loss: 0.0200 - val_accuracy: 0.9360 - val_loss: 0.1299
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 478ms/step - accuracy: 0.9822 - loss: 0.0580 - val_accu

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_18 (Conv1D)              │ (None, 1020, 64)       │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_19 (Conv1D)              │ (None, 1016, 128)      │        41,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_20 (Conv1D)              │ (None, 1012, 256)      │       164,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 259072)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 256)            │    66,322,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

 Trainable params: 689,282 (2.63 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,378,566 (5.26 MB)

Epoch 1: Loss = 16.2899
Epoch 2: Loss = 4.4058
Epoch 3: Loss = 0.8233
Epoch 4: Loss = 0.3501
Epoch 5: Loss = 0.1112
Epoch 6: Loss = 0.0600
Epoch 7: Loss = 0.0590
Epoch 8: Loss = 0.0428
Epoch 9: Loss = 0.0266
Epoch 10: Loss = 0.0252
Epoch 11: Loss = 0.0169
Epoch 12: Loss = 0.0134
Epoch 13: Loss = 0.0175
Epoch 14: Loss = 0.0261
Epoch 15: Loss = 0.0221
Epoch 16: Loss = 0.0112
Epoch 17: Loss = 0.0117
Epoch 18: Loss = 0.0142
Epoch 19: Loss = 0.0087
Epoch 20: Loss = 0.0080
Epoch 21: Loss = 0.0164
Epoch 22: Loss = 0.0415
Epoch 23: Loss = 0.0441
Epoch 24: Loss = 0.0678
Epoch 25: Loss = 0.0475
Epoch 26: Loss = 0.0194
Epoch 27: Loss = 0.0277
Epoch 28: Loss = 0.0152
Epoch 29: Loss = 0.0144
Epoch 30: Loss = 0.0088
Epoch 31: Loss = 0.0051
Epoch 32: Loss = 0.0108
Epoch 33: Loss = 0.0040
Epoch 34: Loss = 0.0069
Epoch 35: Loss = 0.0013
Epoch 36: Loss = 0.0032
Epoch 37: Loss = 0.0015
Epoch 38: Loss = 0.0022
Epoch 39: Loss = 0.0036
Epoch 40: Loss = 0.0046
Epoch 41: Loss = 0.0045
Epoch 42: Loss = 0.0017


: 

In [1]:
import numpy as np
data = np.load('./layer_npz_features/train_10.npz')
features = data['features']
speaker_ids = data['speaker_ids']

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# Load the data

train_data = np.load('./layer_npz_features/train_10.npz')
test_data = np.load('./layer_npz_features/test_10.npz')

train_features = train_data['features']
train_labels = pd.read_csv('y_train.csv').Gender_bin
test_features = test_data['features']
test_labels = pd.read_csv('y_test.csv').Gender_bin

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)

# Transform only the filtered test labels
test_labels_encoded = label_encoder.transform(test_labels)

from sklearn.metrics import classification_report

from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)

svm_model.fit(train_features, train_labels_encoded)

svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# cnn_model = Sequential([
#     Conv1D(32, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
#     MaxPooling1D(pool_size=2),
#     Conv1D(128, kernel_size=5, activation='relu'),
#     MaxPooling1D(pool_size=2),
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(len(label_encoder.classes_), activation='softmax')
# ])

cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(1024, 1)),  # Input shape for CNN
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch+1}: Loss = {logs['loss']:.4f}, Accuracy = {logs['accuracy']:.4f}, Validation Loss = {logs['val_loss']:.4f}, Validation Accuracy = {logs['val_accuracy']:.4f}")

history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2
)

for epoch, (loss, accuracy, val_loss, val_accuracy) in enumerate(zip(
    history.history['loss'],
    history.history['accuracy'],
    history.history['val_loss'],
    history.history['val_accuracy']
)):
    print(f"Epoch {epoch+1}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}, Validation Loss = {val_loss:.4f}, Validation Accuracy = {val_accuracy:.4f}")

cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

from sklearn.metrics import classification_report

# Predict class probabilities
test_predictions_prob = cnn_model.predict(test_features_cnn)

# Get the class with the highest probability
test_predictions = np.argmax(test_predictions_prob, axis=1)

# Generate the classification report
report = classification_report(test_labels_encoded, test_predictions)
print(report)

cnn_model.summary()

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import LabelEncoder, StandardScaler

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# ann_model = Sequential([
#     Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
#     Dropout(0.5),
#     Dense(256, activation='relu'),
#     Dropout(0.5),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(len(label_encoder.classes_), activation='softmax')
# ])

ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(1024,)),  # Input shape for ANN
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2
)

for epoch, (loss, accuracy, val_loss, val_accuracy) in enumerate(zip(
    history.history['loss'],
    history.history['accuracy'],
    history.history['val_loss'],
    history.history['val_accuracy']
)):
    print(f"Epoch {epoch+1}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}, Validation Loss = {val_loss:.4f}, Validation Accuracy = {val_accuracy:.4f}")

ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

ann_model.summary()

import torch
from torch.utils.data import DataLoader, TensorDataset

# Convert NumPy arrays to PyTorch tensors with proper shape
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # (batch_size, sequence_length, channels) -> (batch_size, channels, sequence_length)
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

# Create datasets and data loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Define the CNN model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 1024, 512)  # Adjust based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Create tensors and data loaders
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize model, criterion, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

    def test_model(model, test_loader, criterion):
        model.eval()  # Set the model to evaluation mode
        test_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():  # No need to calculate gradients
            for inputs, labels in test_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                test_loss += loss.item() * inputs.size(0)  # Accumulate loss
                _, predicted = torch.max(outputs, 1)  # Get the index of the max log-probability
                total += labels.size(0)
                correct += (predicted == labels).sum().item()  # Count correct predictions

        avg_loss = test_loss / total
        accuracy = correct / total
        return avg_loss, accuracy


# Test the model
test_loss, test_accuracy = test_model(model, test_loader, criterion)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")



/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


SVM Accuracy: 0.9070
SVM Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.98      0.90        56
           1       0.98      0.85      0.91        73

    accuracy                           0.91       129
   macro avg       0.91      0.92      0.91       129
weighted avg       0.92      0.91      0.91       129



2024-09-06 17:47:36.154188: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-06 17:47:37.028204: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-09-06 17:47:38.196893: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning

Epoch 1/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 12s 492ms/step - accuracy: 0.5838 - loss: 1.6527 - val_accuracy: 0.6105 - val_loss: 0.5965
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 481ms/step - accuracy: 0.7741 - loss: 0.5010 - val_accuracy: 0.8256 - val_loss: 0.3564
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 485ms/step - accuracy: 0.8960 - loss: 0.2658 - val_accuracy: 0.9070 - val_loss: 0.2300
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 484ms/step - accuracy: 0.9399 - loss: 0.1563 - val_accuracy: 0.9593 - val_loss: 0.1452
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 486ms/step - accuracy: 0.9555 - loss: 0.0971 - val_accuracy: 0.8488 - val_loss: 0.3279
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 488ms/step - accuracy: 0.9868 - loss: 0.0475 - val_accuracy: 0.9593 - val_loss: 0.0890
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 485ms/step - accuracy: 0.9782 - loss: 0.0530 - val_accuracy: 0.9186 - val_loss: 0.1756
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 486ms/step - accuracy: 0.9945 - loss: 0.0257 - val_accu

Model: "sequential"

...

 Total params: 2,067,848 (7.89 MB)

 Trainable params: 689,282 (2.63 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,378,566 (5.26 MB)

Epoch 1: Loss = 19.3018
Epoch 2: Loss = 5.5107
Epoch 3: Loss = 0.9682
Epoch 4: Loss = 0.3265
Epoch 5: Loss = 0.1722
Epoch 6: Loss = 0.1247
Epoch 7: Loss = 0.0921
Epoch 8: Loss = 0.0737
Epoch 9: Loss = 0.0438
Epoch 10: Loss = 0.0370
Epoch 11: Loss = 0.0416
Epoch 12: Loss = 0.0251
Epoch 13: Loss = 0.0325
Epoch 14: Loss = 0.0345
Epoch 15: Loss = 0.0353
Epoch 16: Loss = 0.0164
Epoch 17: Loss = 0.0119
Epoch 18: Loss = 0.0059
Epoch 19: Loss = 0.0103
Epoch 20: Loss = 0.0111
Epoch 21: Loss = 0.0130
Epoch 22: Loss = 0.0120
Epoch 23: Loss = 0.0089
Epoch 24: Loss = 0.0136
Epoch 25: Loss = 0.0170
Epoch 26: Loss = 0.0198
Epoch 27: Loss = 0.0160
Epoch 28: Loss = 0.0128
Epoch 29: Loss = 0.0178
Epoch 30: Loss = 0.0191
Epoch 31: Loss = 0.0237
Epoch 32: Loss = 0.0456
Epoch 33: Loss = 0.0555
Epoch 34: Loss = 0.0579
Epoch 35: Loss = 0.0398
Epoch 36: Loss = 0.0454
Epoch 37: Loss = 0.0231
Epoch 38: Loss = 0.0315
Epoch 39: Loss = 0.0330
Epoch 40: Loss = 0.0206
Epoch 41: Loss = 0.0089
Epoch 42: Loss = 0.0070


In [2]:
import numpy as np
data = np.load('./layer_npz_features/train_11.npz')
features = data['features']
speaker_ids = data['speaker_ids']

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# Load the data

train_data = np.load('./layer_npz_features/train_11.npz')
test_data = np.load('./layer_npz_features/test_11.npz')

train_features = train_data['features']
train_labels = pd.read_csv('y_train.csv').Gender_bin
test_features = test_data['features']
test_labels = pd.read_csv('y_test.csv').Gender_bin

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)

# Transform only the filtered test labels
test_labels_encoded = label_encoder.transform(test_labels)

from sklearn.metrics import classification_report

from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)

svm_model.fit(train_features, train_labels_encoded)

svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# cnn_model = Sequential([
#     Conv1D(32, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
#     MaxPooling1D(pool_size=2),
#     Conv1D(128, kernel_size=5, activation='relu'),
#     MaxPooling1D(pool_size=2),
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(len(label_encoder.classes_), activation='softmax')
# ])

cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(1024, 1)),  # Input shape for CNN
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch+1}: Loss = {logs['loss']:.4f}, Accuracy = {logs['accuracy']:.4f}, Validation Loss = {logs['val_loss']:.4f}, Validation Accuracy = {logs['val_accuracy']:.4f}")

history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2
)

for epoch, (loss, accuracy, val_loss, val_accuracy) in enumerate(zip(
    history.history['loss'],
    history.history['accuracy'],
    history.history['val_loss'],
    history.history['val_accuracy']
)):
    print(f"Epoch {epoch+1}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}, Validation Loss = {val_loss:.4f}, Validation Accuracy = {val_accuracy:.4f}")

cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

from sklearn.metrics import classification_report

# Predict class probabilities
test_predictions_prob = cnn_model.predict(test_features_cnn)

# Get the class with the highest probability
test_predictions = np.argmax(test_predictions_prob, axis=1)

# Generate the classification report
report = classification_report(test_labels_encoded, test_predictions)
print(report)

cnn_model.summary()

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import LabelEncoder, StandardScaler

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# ann_model = Sequential([
#     Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
#     Dropout(0.5),
#     Dense(256, activation='relu'),
#     Dropout(0.5),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(len(label_encoder.classes_), activation='softmax')
# ])

ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(1024,)),  # Input shape for ANN
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2
)

for epoch, (loss, accuracy, val_loss, val_accuracy) in enumerate(zip(
    history.history['loss'],
    history.history['accuracy'],
    history.history['val_loss'],
    history.history['val_accuracy']
)):
    print(f"Epoch {epoch+1}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}, Validation Loss = {val_loss:.4f}, Validation Accuracy = {val_accuracy:.4f}")

ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

ann_model.summary()

import torch
from torch.utils.data import DataLoader, TensorDataset

# Convert NumPy arrays to PyTorch tensors with proper shape
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # (batch_size, sequence_length, channels) -> (batch_size, channels, sequence_length)
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

# Create datasets and data loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Define the CNN model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 1024, 512)  # Adjust based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Create tensors and data loaders
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize model, criterion, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

    def test_model(model, test_loader, criterion):
        model.eval()  # Set the model to evaluation mode
        test_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():  # No need to calculate gradients
            for inputs, labels in test_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                test_loss += loss.item() * inputs.size(0)  # Accumulate loss
                _, predicted = torch.max(outputs, 1)  # Get the index of the max log-probability
                total += labels.size(0)
                correct += (predicted == labels).sum().item()  # Count correct predictions

        avg_loss = test_loss / total
        accuracy = correct / total
        return avg_loss, accuracy


# Test the model
test_loss, test_accuracy = test_model(model, test_loader, criterion)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")



SVM Accuracy: 0.8992
SVM Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.96      0.89        56
           1       0.97      0.85      0.91        73

    accuracy                           0.90       129
   macro avg       0.90      0.91      0.90       129
weighted avg       0.91      0.90      0.90       129

Epoch 1/50


/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 12s 493ms/step - accuracy: 0.4963 - loss: 3.0974 - val_accuracy: 0.7733 - val_loss: 0.6279
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 474ms/step - accuracy: 0.7375 - loss: 0.5564 - val_accuracy: 0.8430 - val_loss: 0.3185
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 464ms/step - accuracy: 0.8469 - loss: 0.3324 - val_accuracy: 0.8895 - val_loss: 0.2761
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 462ms/step - accuracy: 0.9049 - loss: 0.2150 - val_accuracy: 0.9419 - val_loss: 0.1969
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 458ms/step - accuracy: 0.9488 - loss: 0.1490 - val_accuracy: 0.9070 - val_loss: 0.1829
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 467ms/step - accuracy: 0.9627 - loss: 0.1047 - val_accuracy: 0.9419 - val_loss: 0.1545
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 465ms/step - accuracy: 0.9789 - loss: 0.1138 - val_accuracy: 0.9360 - val_loss: 0.1623
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 463ms/step - accuracy: 0.9877 - loss: 0.0463 - val_accuracy: 0.912

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_3 (Conv1D)               │ (None, 1020, 64)       │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 1016, 128)      │        41,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 1012, 256)      │       164,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 259072)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │    66,322,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

 Total params: 2,067,848 (7.89 MB)

 Trainable params: 689,282 (2.63 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,378,566 (5.26 MB)

Epoch 1: Loss = 11.1742
Epoch 2: Loss = 2.8030
Epoch 3: Loss = 1.0007
Epoch 4: Loss = 0.3085
Epoch 5: Loss = 0.1528
Epoch 6: Loss = 0.1243
Epoch 7: Loss = 0.0374
Epoch 8: Loss = 0.0451
Epoch 9: Loss = 0.0447
Epoch 10: Loss = 0.0269
Epoch 11: Loss = 0.0200
Epoch 12: Loss = 0.0115
Epoch 13: Loss = 0.0085
Epoch 14: Loss = 0.0059
Epoch 15: Loss = 0.0040
Epoch 16: Loss = 0.0061
Epoch 17: Loss = 0.0043
Epoch 18: Loss = 0.0039
Epoch 19: Loss = 0.0029
Epoch 20: Loss = 0.0035
Epoch 21: Loss = 0.0021
Epoch 22: Loss = 0.0108
Epoch 23: Loss = 0.0096
Epoch 24: Loss = 0.0110
Epoch 25: Loss = 0.0241
Epoch 26: Loss = 0.0269
Epoch 27: Loss = 0.0200
Epoch 28: Loss = 0.0111
Epoch 29: Loss = 0.0097
Epoch 30: Loss = 0.0064
Epoch 31: Loss = 0.0083
Epoch 32: Loss = 0.0094
Epoch 33: Loss = 0.0107
Epoch 34: Loss = 0.0207
Epoch 35: Loss = 0.0127
Epoch 36: Loss = 0.0106
Epoch 37: Loss = 0.0088
Epoch 38: Loss = 0.0162
Epoch 39: Loss = 0.0114
Epoch 40: Loss = 0.0195
Epoch 41: Loss = 0.0150
Epoch 42: Loss = 0.0118


In [3]:
import numpy as np
data = np.load('./layer_npz_features/train_12.npz')
features = data['features']
speaker_ids = data['speaker_ids']

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# Load the data

train_data = np.load('./layer_npz_features/train_12.npz')
test_data = np.load('./layer_npz_features/test_12.npz')

train_features = train_data['features']
train_labels = pd.read_csv('y_train.csv').Gender_bin
test_features = test_data['features']
test_labels = pd.read_csv('y_test.csv').Gender_bin

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)

# Transform only the filtered test labels
test_labels_encoded = label_encoder.transform(test_labels)

from sklearn.metrics import classification_report

from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)

svm_model.fit(train_features, train_labels_encoded)

svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# cnn_model = Sequential([
#     Conv1D(32, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
#     MaxPooling1D(pool_size=2),
#     Conv1D(128, kernel_size=5, activation='relu'),
#     MaxPooling1D(pool_size=2),
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(len(label_encoder.classes_), activation='softmax')
# ])

cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(1024, 1)),  # Input shape for CNN
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch+1}: Loss = {logs['loss']:.4f}, Accuracy = {logs['accuracy']:.4f}, Validation Loss = {logs['val_loss']:.4f}, Validation Accuracy = {logs['val_accuracy']:.4f}")

history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2
)

for epoch, (loss, accuracy, val_loss, val_accuracy) in enumerate(zip(
    history.history['loss'],
    history.history['accuracy'],
    history.history['val_loss'],
    history.history['val_accuracy']
)):
    print(f"Epoch {epoch+1}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}, Validation Loss = {val_loss:.4f}, Validation Accuracy = {val_accuracy:.4f}")

cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

from sklearn.metrics import classification_report

# Predict class probabilities
test_predictions_prob = cnn_model.predict(test_features_cnn)

# Get the class with the highest probability
test_predictions = np.argmax(test_predictions_prob, axis=1)

# Generate the classification report
report = classification_report(test_labels_encoded, test_predictions)
print(report)

cnn_model.summary()

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import LabelEncoder, StandardScaler

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# ann_model = Sequential([
#     Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
#     Dropout(0.5),
#     Dense(256, activation='relu'),
#     Dropout(0.5),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(len(label_encoder.classes_), activation='softmax')
# ])

ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(1024,)),  # Input shape for ANN
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2
)

for epoch, (loss, accuracy, val_loss, val_accuracy) in enumerate(zip(
    history.history['loss'],
    history.history['accuracy'],
    history.history['val_loss'],
    history.history['val_accuracy']
)):
    print(f"Epoch {epoch+1}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}, Validation Loss = {val_loss:.4f}, Validation Accuracy = {val_accuracy:.4f}")

ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

ann_model.summary()

import torch
from torch.utils.data import DataLoader, TensorDataset

# Convert NumPy arrays to PyTorch tensors with proper shape
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # (batch_size, sequence_length, channels) -> (batch_size, channels, sequence_length)
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

# Create datasets and data loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Define the CNN model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 1024, 512)  # Adjust based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Create tensors and data loaders
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize model, criterion, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

    def test_model(model, test_loader, criterion):
        model.eval()  # Set the model to evaluation mode
        test_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():  # No need to calculate gradients
            for inputs, labels in test_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                test_loss += loss.item() * inputs.size(0)  # Accumulate loss
                _, predicted = torch.max(outputs, 1)  # Get the index of the max log-probability
                total += labels.size(0)
                correct += (predicted == labels).sum().item()  # Count correct predictions

        avg_loss = test_loss / total
        accuracy = correct / total
        return avg_loss, accuracy


# Test the model
test_loss, test_accuracy = test_model(model, test_loader, criterion)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")



SVM Accuracy: 0.8837
SVM Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.96      0.88        56
           1       0.97      0.82      0.89        73

    accuracy                           0.88       129
   macro avg       0.89      0.89      0.88       129
weighted avg       0.90      0.88      0.88       129

Epoch 1/50


/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 12s 505ms/step - accuracy: 0.5115 - loss: 2.1094 - val_accuracy: 0.6395 - val_loss: 0.6396
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 488ms/step - accuracy: 0.7183 - loss: 0.5764 - val_accuracy: 0.8547 - val_loss: 0.4233
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 465ms/step - accuracy: 0.8377 - loss: 0.3966 - val_accuracy: 0.9128 - val_loss: 0.3241
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 460ms/step - accuracy: 0.9223 - loss: 0.2423 - val_accuracy: 0.9477 - val_loss: 0.2016
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 465ms/step - accuracy: 0.9284 - loss: 0.1869 - val_accuracy: 0.9419 - val_loss: 0.1565
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 464ms/step - accuracy: 0.9456 - loss: 0.1378 - val_accuracy: 0.9360 - val_loss: 0.1536
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 465ms/step - accuracy: 0.9452 - loss: 0.1218 - val_accuracy: 0.8721 - val_loss: 0.2567
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 464ms/step - accuracy: 0.9818 - loss: 0.0563 - val_accuracy: 0.947

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_6 (Conv1D)               │ (None, 1020, 64)       │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_7 (Conv1D)               │ (None, 1016, 128)      │        41,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_8 (Conv1D)               │ (None, 1012, 256)      │       164,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 259072)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 256)            │    66,322,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

 Total params: 2,067,848 (7.89 MB)

 Trainable params: 689,282 (2.63 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,378,566 (5.26 MB)

Epoch 1: Loss = 11.5522
Epoch 2: Loss = 1.8822
Epoch 3: Loss = 0.7555
Epoch 4: Loss = 0.2740
Epoch 5: Loss = 0.1966
Epoch 6: Loss = 0.1511
Epoch 7: Loss = 0.1153
Epoch 8: Loss = 0.0731
Epoch 9: Loss = 0.0815
Epoch 10: Loss = 0.0673
Epoch 11: Loss = 0.0728
Epoch 12: Loss = 0.0524
Epoch 13: Loss = 0.0372
Epoch 14: Loss = 0.0346
Epoch 15: Loss = 0.0187
Epoch 16: Loss = 0.0167
Epoch 17: Loss = 0.0088
Epoch 18: Loss = 0.0142
Epoch 19: Loss = 0.0147
Epoch 20: Loss = 0.0177
Epoch 21: Loss = 0.0138
Epoch 22: Loss = 0.0269
Epoch 23: Loss = 0.0190
Epoch 24: Loss = 0.0136
Epoch 25: Loss = 0.0402
Epoch 26: Loss = 0.0256
Epoch 27: Loss = 0.0478
Epoch 28: Loss = 0.0459
Epoch 29: Loss = 0.0331
Epoch 30: Loss = 0.1134
Epoch 31: Loss = 0.0697
Epoch 32: Loss = 0.0835
Epoch 33: Loss = 0.0712
Epoch 34: Loss = 0.0576
Epoch 35: Loss = 0.0508
Epoch 36: Loss = 0.0432
Epoch 37: Loss = 0.0356
Epoch 38: Loss = 0.0325
Epoch 39: Loss = 0.0312
Epoch 40: Loss = 0.0595
Epoch 41: Loss = 0.1123
Epoch 42: Loss = 0.0670
